In [272]:
import yaml

In [273]:
import csv

In [754]:
import datetime
import re
import os
from ruamel.yaml import YAML
yaml = YAML(typ='safe')
yaml.default_flow_style = False
yaml.representer.ignore_aliases = lambda x: True

def iter_files():
    outdir = 'engagements'
    for filename in os.listdir(outdir):
        fn = os.path.join(outdir, filename)
        state, ext = os.path.splitext(fn)
        state = state.split("/")[1]
        if ext == '.yml':
            with open(fn, 'r') as fp:
                data = yaml.load(fp)
            yield (state, data)

In [755]:
dyer = dict(iter_files())

- force name
- detachment
- type

In [804]:
UNIT_ORGS = [
  ("ALABAMA", Optional(("and", "TENNESSEE"))),
 'ARKANSAS',
 'CALIFORNIA',
 'COLORADO',
 'CONNECTICUT',
 'DAKOTA',
 'DELAWARE',
 'DISTRICT OF COLUMBIA',
 'FLORIDA',
 'IDAHO TERRITORY',
 'ILLINOIS',
 ("INDIANA", Optional("LEGION")),
 'IOWA',
 'KANSAS',
 'KENTUCKY',
 'LOUISIANA',
 'MAINE',
 'MARYLAND',
 'MASSACHUSETTS',
 'MICHIGAN',
 'MINNESOTA',
 ('MISSISSIPPI', Optional(("MARINE", "BRIGADE"))),
 'MISSOURI',
 'NEBRASKA',
 'NEVADA',
 'NEW HAMPSHIRE',
 'NEW JERSEY',
 'NEW MEXICO',
 'NEW YORK',
 'NORTH CAROLINA',
 'OHIO',
 'OREGON',
 'PENNSYLVANIA',
 'RHODE ISLAND',
 'SOUTH CAROLINA',
 'TENNESSEE',
 'TEXAS',
 ('UNITED', 'STATES', Optional(["NAVY", ("COLORED", "TROOPS")])),
 'VERMONT',
 'VIRGINIA',
 'WASHINGTON',
 'WEST VIRGINIA',
 'WISCONSIN']

In [805]:
from arpeggio import Optional, ZeroOrMore, OneOrMore, EOF, UnorderedGroup, OrderedChoice, Sequence
from arpeggio import RegExMatch as _
from arpeggio import ParserPython

def ordinal():
    return _('\d+(st|nd|rd|th)')

def punct():
    return _("[.,;:]")

def integer():
    return _("\d+")

def list_sep():
    return [(",", "and"), ",", "and"]


Test unit type parser

In [806]:
SHIP_NAMES = [
    "Conestoga", 
    "Lexington",
    "Tyler",
    "Brooklyn", 
    "Octorora", 
    "Hartford", 
    "Ossippee",
    "Itasca", 
    "Oneida", 
    "Galena", 
    "Metacomet", 
    "Richmond", 
    "Port Royal", 
    "Lackawanna",
    "Seminole", 
    "Monongahela",
    "Tecumseh"
]

def ship_name():
    return Optional("\""), SHIP_NAMES, Optional(punct), Optional("\"")

def ships():
    """Lists of Ships
    
    - U. S. Gunboats "Conestoga", "Lexington" and "Tyler."
    
    """
    return _(r"U.\s*S.\s+Gunboats?"), Optional(ship_name, ZeroOrMore(Optional(list_sep), ship_name))

In [807]:
ParserPython(ships).parse("U. S. Gunboats")

[  'U. S. Gunboats' [0] ]

In [808]:
ParserPython(ships).parse("U. S. Gunboats \"Conestoga\", \"Lexington\" and \"Tyler.\"")

[  'U. S. Gunboats' [0], [  '"' [15],  'Conestoga' [16],  '"' [25] ], [  ',' [26] ], [  '"' [28],  'Lexington' [29],  '"' [38] ], [  'and' [40] ], [  '"' [44],  'Tyler' [45], punct '.' [50],  '"' [51] ] ]

In [809]:
OTHER_PATTERNS = [
    "By State Troops",
    "Gen. R. L. McCook and Escort. Gen. McCook killed.",
    "(No Reports.)",
    "(Confederate Reports.)",
    "Train Guard",
    "Naval Attack on Blockade Runner",
    "(No Details.)",
    "Scouting party",
    "Foraging party",
    "By Confederate Forces",
    "To U. S. Forces",
    "Surrender of Confederate Forces in Departments of Alabama, Mississippi and Eastern Louisiana.",
    "Explosion of Ordnance Depot.",
    "United States forces",
    _("Attack on U. S. Steamer .*"),
    "(See Force in Campaign against Mobile.)",
    "13th Army Corps.",
    "Picket Attack.",
    "Pickets",
    "Pickets, 2nd Brigade, 1st Division, 16th Corps"
]

def quoted_letter():
    return "\"", _("[A-Z]"), Optional(punct), "\""

def other():
    return OTHER_PATTERNS

def detachment():
    """Detachment pattern.
    
    - (Detachment)
    
    """
    return "(", "Detachment", ")"

def detachments():
    """Detachments pattern
    
    - (Detachments)
    
    """
    return "(", "Detachments", ")"

def section():
    return "(", "Section", ")"

def battalion():
    return "(", Optional(ordinal), "Battalion", ")"

def mounted():
    return "(", "Mounted", ")"

print(ParserPython(detachment).parse("(Detachment)"))
print(ParserPython(section).parse("(Section)"))

def named_companies():
    """List of Company Names.
    
    - (Co. "A")
    - (Cos. "A" and "B")
    - (Cos. "A", "B", and "C")
    
    """
    return "(", _("Cos?\.?"), quoted_letter, ZeroOrMore(list_sep, quoted_letter), ")"

def num_companies():
    """Number of companies
    
    - (4 Cos.)
    - (1 Co.)
    
    """
    return "(", integer, _("Cos?\.?"), ")"

def companies():
    return [named_companies, num_companies]


print(ParserPython(named_companies).parse('(Cos. "A")'))
print(ParserPython(named_companies).parse('(Cos. "A", "B", and "C")'))
print(ParserPython(num_companies).parse('(4 Cos.)'))


( | Detachment | )
( | Section | )
( | Cos. | " | A | " | )
( | Cos. | " | A | " | , | " | B | " | , | and | " | C | " | )
( | 4 | Cos. | )


In [773]:
def one_battery():
    return "Battery", quoted_letter

def multiple_batteries():
    return "Batteries", OneOrMore(Optional(list_sep), quoted_letter)

def artillery():
    """Artillery Batteries.
        
    - "Battery \"B\", 1st Light Artillery (Section)"
    
    """
    return (Optional("Independent"),
            [one_battery, multiple_batteries], 
            Optional(punct), 
            Optional(ordinal), 
            Optional("Light"), "Artillery",
            Optional([section, detachment])
           )

ex = "Battery \"B\", 1st Light Artillery"
ParserPython(artillery_battery).parse(ex)
ex = "Battery \"B\", 1st Light Artillery (Section)"
print(ParserPython(artillery_battery).parse(ex))

Battery | " | B | " | , | 1st | Light | Artillery | ( | Section | )


In [793]:
UNIT_TYPES = [(Optional(["Middle", "East", "Veteran", "Vidette"]), "Cavalry"),
              (Optional(["Mounted", "Veteran", "Colored"]), "Infantry"),
              (Optional("Independent"), Optional(["Battery", "Batteries"]), "Light", "Artillery"),
              (Optional("Heavy"), "Artillery")
             ]

def ordinal_unit():
    return (ordinal, 
            UNIT_TYPES,
            Optional([detachment, battalion, named_companies, num_companies, mounted])
            )

def ordinal_unit_in_list():
    return ordinal, Optional([mounted, companies])
    
def ordinal_units():
    return (ordinal_unit_in_list, 
            ZeroOrMore(Optional(list_sep), 
                       ordinal_unit_in_list), 
            UNIT_TYPES,
            Optional(detachments))

for i in ("1st, 2nd, and 4th Infantry", 
          "22nd and 44th Cavalry",
          "6th and 12th Independent Batteries Light Artillery"
         ):
    print(ParserPython(ordinal_units).parse(i))

1st | , | 2nd | , | and | 4th | Infantry
22nd | and | 44th | Cavalry
6th | and | 12th | Independent | Batteries | Light | Artillery


In [799]:
OTHER_UNIT_TEXT = [
    "Navy",
    "Batteries"
]

NAMED_UNITS = [
    ("Landgraeber's", "Battery", '"F"', Optional(punct), "2nd", "Light", "Artillery"),
    "Landgraeber's Battery Flying Artillery",
    "Landgraeber's Battery Flying Artillery ("F" 2nd Artillery)",    
    ("Cogswell's", "Independent", "Battery", "Light", "Artillery"),
    ("Latham's", "Co.", "Cavalry"),
    "Chicago Board of Trade Battery Light Artillery",
    "Fremont's Hussars",
    "Benton Hussars",
    "Bowen's Battalion Cavalry",
    "Jenks' and Smith's Cavalry Cos.",
    "Phelps' Regt. Infantry",
    "5th Co. Sharpshooters.", # Ohio
]

def unit_name():
    """Single unit name
    
    - 1st Cavalry (Detachment)
    - 5th Cavalry
    - Battery "B", 1st Light Artillery
    
    """
    return [NAMED_UNITS,
            artillery,
            ordinal_unit,
            ordinal_units,
            OTHER_UNIT_TEXT
           ]

def unit_list():
    """Lists of Units.
    
    - KENTUCKY--5th Cavalry
    - PENNSYLVANIA--7th Cavalry (Detachment); 78th and 79th Infantry
    
    """
    return UNIT_ORGS, _('-+'), (OneOrMore(unit_name, Optional(punct)))

def union_losses():
    return _("(Union loss|Loss).*")


Miscellaneous stuff after the troops engaged:

In [800]:
extra_text = [
    "picket attack",
    "(Destruction of Salt Works.)"
]

In [801]:
def troops_engaged():
    return ([other, 
             ships, 
             OneOrMore(unit_list)], 
            Optional(punct), 
            Optional(extra_text), 
            Optional(punct), 
            Optional(union_losses), 
            EOF)

In [810]:
from arpeggio import NoMatch
parser = ParserPython(troops_engaged)
for engagement in dyer["Arkansas"]:
    try:
        text = engagement['troops_engaged']
    except KeyError:
        continue
    try:
        parser.parse(text)
    except NoMatch:
        print(engagement['slug'], text)
    

Arkansas/1861-12-09-skirmish-bushy-creek Union Indians, under Opothleyholo.
Arkansas/1862-02-16-action-pott-s-hill-sugar-creek ILLINOIS--3rd Cavalry. MISSOURI--Wright's Battalion Cavalry. Union loss, 1 killed, 3 wounded. Total, 4.
Arkansas/1862-05-19-skirmish-searcy-landing-little-red-river MISSOURI--4th Cavalry (Cos. "A," "B," "C," "D"); Battery "B," 1st Light Artillery; 3rd (Co. "B") and 17th Infantry (Cos. "A," "F," "G," "H"). Union loss, 15 killed, 32 wounded, 2 missing. Total, 49.
Arkansas/1862-06-12-action-waddell-s-farm-indian-creek ILLINOIS--9th Cavalry (Cos. "A," "C," "K," "M"). IOWA--3rd Cavalry. MISSOURI--Bowen's Battalion Cavalry. Union loss, 4 killed, 8 wounded. Total, 12.
Arkansas/1862-06-17-engagement-st-charles-white-river INDIANA--46th Infantry. UNITED STATES--Navy; Gunboats "Lexington," "Conestoga," "Mound City" and "St. Louis". Union loss, 30 killed, 105 wounded. Total, 135.
Arkansas/1862-06-17-skirmish-smithville ILLINOIS--5th Cavalry (Cos. "D," "F," "L"). Union los